In [16]:
!pip install openai

In [17]:
import openai
import json
import re
from typing import List, Dict
from google.colab import userdata

# Set your API key (input this manually)
key = userdata.get('OPENAI_API_KEY')
openai.api_key = key

schema_english = {
    "general information": {
        "name": "string",
        "brand": "string",
        "category": "string",
        "original_price": "float",
        "discount_price": "float",
        "description": "string",
        "design": "string",
        "promotion": "string",
    },
    "technical specifications": {
        "display": ["string"],
        "memory and storage": ["string"],
        "camera": "string",
        "battery": "string",
        "operating system": "string",
        "chip": "string",
        "connectivity": ["string"],
        "audio": "string",
        "charging and expansion": ["string"],
    },
    "extra_metadata": {
        "special feature": ["string"],
        "accessories": ["string"],
    }
}


def extract_and_clean_json_from_response(content: str) -> str:
    """Extract JSON data from a response and remove comments."""
    # Extract JSON content between ```json and ```
    json_match = re.search(r'```json\n(.*?)\n```', content, re.DOTALL)
    if json_match:
        json_content = json_match.group(1)

        # Remove any comments or lines with `//`
        cleaned_json_content = re.sub(r'//.*', '', json_content)
        return cleaned_json_content
    return None


def convert_to_json(raw_data, schema):
    # Define the prompt
    raw_data = raw_data.strip().rstrip('.').rstrip('...').strip()
    prompt = f"Convert the following raw data into JSON format based on this schema:\n\nSchema: {json.dumps(schema)}\n\nRaw Data: {json.dumps(raw_data)}"

    # Call the OpenAI API
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that formats data."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
    )

    # Extract and parse the JSON content
    content = response.choices[0].message.content.strip()
    json_content = extract_and_clean_json_from_response(content)

    if json_content:
        try:
            json_data = json.loads(json_content)  # Parse the JSON-formatted text
            return json_data
        except json.JSONDecodeError as e:
            print("Error decoding JSON:", e)
    else:
        print("No JSON found in response.")
    return None

def process_and_save_data(raw_data_list, output_file):
    schema = schema_english

    processed_data = []

    for raw_data in raw_data_list:
        json_data = convert_to_json(raw_data, schema)
        if json_data:
            processed_data.append(json_data)

    # Save the processed data to a JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(processed_data, f, ensure_ascii=False, indent=4)

# # Example usage with a list of raw data strings
# raw_data_list = [
#     "Tên sản phẩm: iPhone 13, Thương hiệu: Apple, Giá: 20000000, Danh mục: điện thoại thông minh",
#     "Tên sản phẩm: Galaxy S21, Thương hiệu: Samsung, Giá: 18000000, Danh mục: điện thoại thông minh",
#     "Tên sản phẩm: Xiaomi Mi 11, Thương hiệu: Xiaomi, Giá: 15000000, Danh mục: điện thoại thông minh",
#     "Tên sản phẩm: Oppo Find X3 Pro, Thương hiệu: Oppo, Giá: 22000000, Danh mục: điện thoại thông minh"
# ]

# # Save converted data to 'output_data.json'
# process_and_save_data(raw_data_list, 'output_data.json')


In [ ]:
# import json

# # Open and read the JSON file
# with open('output_data.json', 'r') as file:
#     data = json.load(file)

# # Print the data
# print(data)


[{'general information': {'name': 'iPhone 13', 'brand': 'Apple', 'category': 'điện thoại thông minh', 'price': 20000000.0, 'description': '', 'design': ''}, 'technical specifications': {'display': [], 'memory and storage': [], 'camera': '', 'battery': '', 'operating system': '', 'chip': '', 'connectivity': [], 'audio': '', 'charging and expansion': []}, 'extra_metadata': {'special feature': [], 'accessories': []}}, {'general information': {'name': 'Galaxy S21', 'brand': 'Samsung', 'category': 'điện thoại thông minh', 'price': 18000000.0, 'description': '', 'design': ''}, 'technical specifications': {'display': [], 'memory and storage': [], 'camera': '', 'battery': '', 'operating system': '', 'chip': '', 'connectivity': [], 'audio': '', 'charging and expansion': []}, 'extra_metadata': {'special feature': [], 'accessories': []}}, {'general information': {'name': 'Xiaomi Mi 11', 'brand': 'Xiaomi', 'category': 'điện thoại thông minh', 'price': 15000000.0, 'description': '', 'design': ''}, 

# Raw Data:

In [ ]:
import requests
from bs4 import BeautifulSoup

def spider(max_pages):
    page = 1
    while page <= max_pages:
        url = 'https://hoanghamobile.com/dien-thoai-di-dong?p=' + str(page)
        source_code = requests.get(url)
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'html.parser')

        print("Products List:")
        print("================================================")
        # Tìm tất cả các thẻ <div> có class "pinfo" và trong đó có thẻ <h3>
        for pinfo in soup.find_all('div', class_='v5-item'):
            # Lấy thẻ <a> trong thẻ <h3> để lấy link
            link_tag = pinfo.find('h3').find('a')
            price_tag = pinfo.find('div', class_='price')
            href = 'https://hoanghamobile.com/' + link_tag.get('href')
            title = link_tag.get('title')
            print(f"Product Title: {title}")
            print(f"Product URL: {href}")
            print()

        page += 1

# Chạy hàm để kiểm tra
spider(1)  # Bạn có thể thay đổi số trang nếu muốn lấy nhiều hơn


Products List:
Product Title: Samsung Galaxy Z Fold6 12GB/256GB
Product URL: https://hoanghamobile.com//dien-thoai/samsung-galaxy-z-fold6

Product Title: Samsung Galaxy Z Fold6 12GB/512GB
Product URL: https://hoanghamobile.com//dien-thoai/samsung-galaxy-z-fold6-12gb-512gb

Product Title: Samsung Galaxy Z Fold6 12GB/1TB
Product URL: https://hoanghamobile.com//dien-thoai/samsung-galaxy-z-fold6-12gb-1tb

Product Title: Samsung Galaxy Z Flip6 12GB/256GB
Product URL: https://hoanghamobile.com//dien-thoai/samsung-galaxy-z-flip6

Product Title: Samsung Galaxy Z Flip6 12GB/512GB
Product URL: https://hoanghamobile.com//dien-thoai/samsung-galaxy-z-flip6-12gb-512gb

Product Title: iPhone 16 Pro Max - Chính hãng VN/A
Product URL: https://hoanghamobile.com//dien-thoai/iphone-16-pro-max

Product Title: iPhone 16 - Chính hãng VN/A
Product URL: https://hoanghamobile.com//dien-thoai/iphone-16

Product Title: iPhone 16 Pro - Chính hãng VN/A
Product URL: https://hoanghamobile.com//dien-thoai/iphone-16-pr

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

def spider(max_pages):
    page = 1
    while page <= max_pages:
        url = 'https://hoanghamobile.com/dien-thoai-di-dong?p=' + str(page)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        print("Products List:")
        print("=================================================")

        # Lặp qua từng sản phẩm
        for pinfo in soup.find_all('div', class_='v5-item'):
            # Lấy link sản phẩm
            link_tag = pinfo.find('h3').find('a')
            href = 'https://hoanghamobile.com' + link_tag.get('href')
            title = link_tag.get('title')

            # Lấy giá hiện tại và giá gốc của sản phẩm
            price_tag = pinfo.find('div', class_='price')
            current_price = price_tag.find('strong').text.strip() if price_tag.find('strong') else 'N/A'
            original_price = price_tag.find('strike').text.strip() if price_tag.find('strike') else 'N/A'

            # In thông tin sản phẩm
            print(f"Product Title: {title}")
            print(f"Product URL: {href}")
            print("Current Price:", current_price)
            print("Original Price:", original_price)
            print("-" * 30)

        # Thêm độ trễ giữa các request để tránh bị chặn
        time.sleep(2)  # Đợi 2 giây giữa các request
        page += 1

# Chạy hàm để kiểm tra
spider(1)  # Thay đổi số trang tùy ý nếu bạn muốn lấy thêm sản phẩm


Products List:
Product Title: Samsung Galaxy Z Fold6 12GB/256GB
Product URL: https://hoanghamobile.com/dien-thoai/samsung-galaxy-z-fold6
Current Price: 41,990,000 ₫
Original Price: 43,990,000 ₫
------------------------------
Product Title: Samsung Galaxy Z Fold6 12GB/512GB
Product URL: https://hoanghamobile.com/dien-thoai/samsung-galaxy-z-fold6-12gb-512gb
Current Price: 45,990,000 ₫
Original Price: 47,990,000 ₫
------------------------------
Product Title: Samsung Galaxy Z Fold6 12GB/1TB
Product URL: https://hoanghamobile.com/dien-thoai/samsung-galaxy-z-fold6-12gb-1tb
Current Price: 52,990,000 ₫
Original Price: 54,990,000 ₫
------------------------------
Product Title: Samsung Galaxy Z Flip6 12GB/256GB
Product URL: https://hoanghamobile.com/dien-thoai/samsung-galaxy-z-flip6
Current Price: 26,990,000 ₫
Original Price: 28,990,000 ₫
------------------------------
Product Title: Samsung Galaxy Z Flip6 12GB/512GB
Product URL: https://hoanghamobile.com/dien-thoai/samsung-galaxy-z-flip6-12gb-

## Testing:

In [1]:
import requests

# URL to send the request
url = "https://hoanghamobile.com/Ajax/fullspecs2/5568"

# Headers to include in the request
headers = {
    "accept": "/",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "en-US,en;q=0.9",
    "referer": "https://hoanghamobile.com/dien-thoai/realme-note-60-4-64gb",
    "sec-ch-ua": '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
    "x-kl-kis-ajax-request": "Ajax_Request",
    "x-requested-with": "XMLHttpRequest"
}

# Sending the GET request
response = requests.get(url, headers=headers)

# Check the response status
if response.status_code == 200:
    # Print the response content
    print("Response content:")
    print(response.text)
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

Response content:



<div class=" box-technical-specifications">
    <div class="item-site-banner">
        <div class="site-banner-content">
            <i class="icon-SettingSolidOff"></i>
            <div class="description">
                <span>
                    THÔNG SỐ KỸ THUẬT
                </span>
            </div>
        </div>
    </div>
    <div class="text-align-start d-block box-specs-content">

            <div class="specs-item">
                <strong class="title">M&#224;n h&#236;nh</strong>
                <ul class="specs-content">
                        <li>
                            <strong>C&#244;ng nghệ m&#224;n h&#236;nh</strong>
                            <span>IPS LCD</span>
                        </li>
                        <li>
                            <strong>Tần số qu&#233;t (Hz)</strong>
                            <span>90Hz</span>
                        </li>
                        <li>
                            <strong>Độ ph&#22

In [3]:
import requests
from bs4 import BeautifulSoup

def get_single_item_metadata(item_url):
    # Gửi yêu cầu tới trang sản phẩm
    headers = {
        "accept": "/",
        "accept-encoding": "gzip, deflate, br, zstd",
        "accept-language": "en-US,en;q=0.9",
        "referer": item_url,
        "sec-ch-ua": '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
        "x-kl-kis-ajax-request": "Ajax_Request",
        "x-requested-with": "XMLHttpRequest"
    }

    response = requests.get(item_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Tìm URL của trang cấu hình chi tiết
    ajax_link_tag = soup.find('a', class_='ajax-modal-show')
    if not ajax_link_tag:
        print("Không tìm thấy liên kết cấu hình chi tiết.")
        return

    # Tạo URL đầy đủ cho yêu cầu AJAX
    ajax_url = "https://hoanghamobile.com" + ajax_link_tag['href']
    # print("URL cấu hình chi tiết:", ajax_url)

    # Gửi yêu cầu tới URL cấu hình chi tiết với headers
    ajax_response = requests.get(ajax_url, headers=headers)
    ajax_soup = BeautifulSoup(ajax_response.text, 'html.parser')

    # Tạo dictionary để lưu trữ cấu hình chi tiết
    specs_data = {}

    # Duyệt qua từng mục specs-item trong trang cấu hình chi tiết
    for specs_item in ajax_soup.find_all('div', class_='specs-item'):
        category_title = specs_item.find('strong', class_='title').get_text(strip=True)
        specs_data[category_title] = {}

        for li in specs_item.find('ul', class_='specs-content').find_all('li'):
            attribute_name = li.find('strong').get_text(strip=True)
            attribute_value = li.find('span').get_text(separator=' ', strip=True)
            specs_data[category_title][attribute_name] = attribute_value

    # In cấu hình đầy đủ
    for category, attributes in specs_data.items():
        print(f"\n{category}:")
        for attr_name, attr_value in attributes.items():
            print(f"  {attr_name}: {attr_value}")

    return specs_data

# Ví dụ sử dụng hàm với URL của một sản phẩm
get_single_item_metadata('https://hoanghamobile.com/dien-thoai/samsung-galaxy-z-fold6')



Màn hình:
  Công nghệ màn hình: Dynamic AMOLED 2X
  Kích thước màn hình: MH chính: 7.6" MH phụ: 6.3"
  Độ sáng màn hình: 2600nits
  Tần số quét (Hz): 1-120Hz
  Độ phân giải: MH chính: 2160 x 1856 MH phụ: 2376 x 968

Camera sau:
  Quay phim: UHD 8K (7680 x 4320)@30fps
  Tính năng: AI Camera với công nghệ ProVisual Engine (AI Zoom, Chụp chân dung AI) Tính năng chụp đêm Nightography Zoom quang học 3x Zoom kỹ thuật số 30x
  Đèn Flash: Có
  Độ phân giải camera: 50 MP (f/1.8) + 12 MP (f/2.2) + 10 MP (f/2.4)

Camera trước:
  Độ phân giải camera: Camera trước ở màn hình phụ: 10 MP Camera ẩn dưới màn hình: 4 MP
  Tính năng: Xóa phông Các tính năng chụp hình thông minh khác

Hệ điều hành & CPU:
  Hệ điều hành: Android 14
  Vi xử lý đồ họa (GPU): Adreno 750
  Vi xử lý: Snapdragon 8 Gen 3
  Tốc độ CPU: 8-core (1x3.39GHz Cortex-X4 & 3x3.1GHz Cortex-A720 & 2x2.9GHz Cortex-A720 & 2x2.2GHz Cortex-A520)

Bộ nhớ & Lưu trữ:
  RAM: 12GB
  Bộ nhớ còn lại (khả dụng): 216.1GB
  Bộ nhớ trong: 256GB
  Thẻ nhớ

{'Màn hình': {'Công nghệ màn hình': 'Dynamic AMOLED 2X',
  'Kích thước màn hình': 'MH chính: 7.6" MH phụ: 6.3"',
  'Độ sáng màn hình': '2600nits',
  'Tần số quét (Hz)': '1-120Hz',
  'Độ phân giải': 'MH chính: 2160 x 1856 MH phụ: 2376 x 968'},
 'Camera sau': {'Quay phim': 'UHD 8K (7680 x 4320)@30fps',
  'Tính năng': 'AI Camera với công nghệ ProVisual Engine (AI Zoom, Chụp chân dung AI) Tính năng chụp đêm Nightography Zoom quang học 3x Zoom kỹ thuật số 30x',
  'Đèn Flash': 'Có',
  'Độ phân giải camera': '50 MP (f/1.8) + 12 MP (f/2.2) + 10 MP (f/2.4)'},
 'Camera trước': {'Độ phân giải camera': 'Camera trước ở màn hình phụ: 10 MP Camera ẩn dưới màn hình: 4 MP',
  'Tính năng': 'Xóa phông Các tính năng chụp hình thông minh khác'},
 'Hệ điều hành & CPU': {'Hệ điều hành': 'Android 14',
  'Vi xử lý đồ họa (GPU)': 'Adreno 750',
  'Vi xử lý': 'Snapdragon 8 Gen 3',
  'Tốc độ CPU': '8-core (1x3.39GHz Cortex-X4 & 3x3.1GHz Cortex-A720 & 2x2.9GHz Cortex-A720 & 2x2.2GHz Cortex-A520)'},
 'Bộ nhớ & Lưu t

In [7]:
import requests
from bs4 import BeautifulSoup

def get_single_desc_item_data(item_url):
    # Gửi yêu cầu tới trang sản phẩm
    source_code = requests.get(item_url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')

    # Tìm phần tử chứa mô tả sản phẩm
    product_desc_tag = soup.find('div', class_='product-content-text box-content-text')
    if not product_desc_tag:
        print("Không tìm thấy mô tả sản phẩm.")
        return None

    # Lấy và trả về nội dung text của mô tả
    product_desc_text = product_desc_tag.get_text(separator=" ", strip=True)
    return product_desc_text

# Ví dụ sử dụng hàm với URL của một sản phẩm
desc = get_single_desc_item_data('https://hoanghamobile.com/dien-thoai/samsung-galaxy-z-fold6')
print("Mô tả sản phẩm:")
print(desc)


Mô tả sản phẩm:
Samsung Galaxy Z Fold6 là dòng điện thoại thông minh thế hệ Galaxy Z Fold tiếp theo được Samsung công bố ra thị trường với nhiều tính năng và cập nhật lớn. Phiên bản này sẽ có màn hình ngoài 6.3 inch , PIN 44 00mAh , và camera trước 10 MP . Galaxy Z Fold6 còn được trang bị hệ thống làm mát lớn hơn với công nghệ chipset Snapdragon 8 Gen 3 For Galaxy cực mạnh. Kèm với đó, điện thoại sẽ có bộ nhớ lên tới 12GB RAM và được trang bị Galaxy AI. Lưu ý: Những thông tin trong bài viết mang tính chất tham khảo. Thông tin chính xác nhất sẽ được cập nhật liên tục theo dòng sự kiện ra mắt của sản phẩm. Đặc điểm nổi bật: Diện mạo mới hiện đại, thanh lịch với phần viền gấp được làm mờ đi so với các thế hệ trước. Nâng cấp lớn về camera với độ phân giải lên đến 50MP, mang lại trải nghiệm chụp ảnh chuyên nghiệp trên điện thoại. Trang bị công nghệ chipset Snapdragon 8 Gen 3 For Galaxy mạnh mẽ, đảm bảo hiệu suất hoạt động của máy. Dung lượng PIN 4400 mAh cải thiện đáng kể năng lượng so với 

In [9]:
import requests
from bs4 import BeautifulSoup

def get_single_promotion_data(item_url):
    # Gửi yêu cầu tới trang sản phẩm
    source_code = requests.get(item_url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')

    # Khởi tạo giá trị mặc định cho khuyến mãi
    promotion_text = "N/A"

    # Tìm phần tử chứa khuyến mãi qua class và id
    promotion_section = soup.find('div', class_='box-promotion-content box-content-text no-after') or \
                        soup.find('div', id='product-promotion-more')

    if promotion_section:
        # Tìm tất cả các phần tử khuyến mãi trong `promotion_section`
        promotion_items = promotion_section.find_all('div', class_='promotion-item')

        # Nếu tìm thấy khuyến mãi, lấy nội dung và nối thành chuỗi
        if promotion_items:
            promotion_text = "; ".join(
                item.find('span').get_text(separator=" ", strip=True)
                for item in promotion_items if item.find('span')
            )

    return promotion_text

# Ví dụ sử dụng hàm với URL của một sản phẩm
promotions = get_single_promotion_data('https://hoanghamobile.com/dien-thoai/samsung-galaxy-z-fold6')
print("Các chương trình khuyến mãi:")
print(promotions)


Các chương trình khuyến mãi:
Trả góp tới 06 tháng không lãi suất, trả trước 0 đồng với Samsung Finance+. (Xem chi tiết); Trả góp 0% - trả trước 0Đ - Từ 1.516.000VNĐ/tháng (Xem chi tiết); Đặc quyền HS-SV: Tặng sim năm sinh (2k4-2k6) khi mua kèm điện thoại, máy tính bảng, laptop, màn hình. (Xem chi tiết); Từ ngày 01-30/11 Giảm 10% khi khách hàng mua phụ kiện kèm Phone + Tablet Apple, Samsung; Giảm 50% tối đa 700k khi mở thẻ tín dụng Vpbank trên SenID (Xem chi tiết); Giảm tới 500.000đ khi thanh toán thẻ tín dụng NCB (Xem chi tiết); Mở thẻ tín dụng VIB - Nhận Voucher 600.000đ (Xem chi tiết); Miễn phí chuyển đổi trả góp hoặc giảm đến 500.000đ khi mở thẻ TPBank EVO (Xem chi tiết); Ưu đãi trả góp 0% qua Homecredit, Shinhan Finance, Mirae asset, Homepaylater, Kredivo (Xem chi tiết); Giảm đến 150.000đ khi thanh toán qua Muadee (Xem chi tiết); Giảm thêm 100.000đ cho tất cả các sản phẩm màn hình khi mua kèm Laptop, MacBook, Máy tính bảng và Điện thoại.


# Save Data to Pandas Dataframe:
```
  --Id
  --Title
  --Original Price
  --Discount
  --Meta_Data
  --Description
```

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def get_single_item_metadata(item_url):
    # Gửi yêu cầu tới trang sản phẩm
    headers = {
        "accept": "/",
        "accept-encoding": "gzip, deflate, br, zstd",
        "accept-language": "en-US,en;q=0.9",
        "referer": item_url,
        "sec-ch-ua": '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
        "x-kl-kis-ajax-request": "Ajax_Request",
        "x-requested-with": "XMLHttpRequest"
    }

    response = requests.get(item_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Tìm URL của trang cấu hình chi tiết
    ajax_link_tag = soup.find('a', class_='ajax-modal-show')
    if not ajax_link_tag:
        return "N/A"

    # Tạo URL đầy đủ cho yêu cầu AJAX
    ajax_url = "https://hoanghamobile.com" + ajax_link_tag['href']
    ajax_response = requests.get(ajax_url, headers=headers)
    ajax_soup = BeautifulSoup(ajax_response.text, 'html.parser')

    # Tạo dictionary để lưu trữ cấu hình chi tiết
    specs_data = {}
    for specs_item in ajax_soup.find_all('div', class_='specs-item'):
        category_title = specs_item.find('strong', class_='title').get_text(strip=True)
        specs_data[category_title] = {}
        for li in specs_item.find('ul', class_='specs-content').find_all('li'):
            attribute_name = li.find('strong').get_text(strip=True)
            attribute_value = li.find('span').get_text(separator=' ', strip=True)
            specs_data[category_title][attribute_name] = attribute_value

    return specs_data

def get_single_desc_item_data(item_url):
    response = requests.get(item_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    product_desc_tag = soup.find('div', class_='product-content-text box-content-text')
    return product_desc_tag.get_text(separator=" ", strip=True) if product_desc_tag else "N/A"

def get_single_promotion_data(item_url):
    response = requests.get(item_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    promotion_section = soup.find('div', class_='box-promotion-content box-content-text no-after') or \
                        soup.find('div', id='product-promotion-more')
    if promotion_section:
        promotion_items = promotion_section.find_all('div', class_='promotion-item')
        return "; ".join(
            item.find('span').get_text(separator=" ", strip=True)
            for item in promotion_items if item.find('span')
        )
    return "N/A"

def clean_price(price_str):
    """Chuyển đổi chuỗi giá thành số nguyên."""
    return int(price_str.replace(',', '').replace('₫', '').strip()) if price_str else 0

def spider(max_pages):
    product_data = []  # List to store all product data
    page = 1
    id_counter = 1

    while page <= max_pages:
        url = 'https://hoanghamobile.com/dien-thoai-di-dong?p=' + str(page)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        for pinfo in soup.find_all('div', class_='v5-item'):
            # Lấy link và tiêu đề sản phẩm
            link_tag = pinfo.find('h3').find('a')
            href = 'https://hoanghamobile.com' + link_tag.get('href')
            title = link_tag.get('title')

            # Lấy giá hiện tại và giá gốc của sản phẩm
            price_tag = pinfo.find('div', class_='price')
            current_price = price_tag.find('strong').text.strip() if price_tag.find('strong') else '0đ'
            original_price = price_tag.find('strike').text.strip() if price_tag.find('strike') else current_price

            # Chuyển đổi giá từ chuỗi sang số nguyên
            current_price_value = clean_price(current_price)
            original_price_value = clean_price(original_price)

            # Tính discount price
            discount_price_value = original_price_value - current_price_value

            # Gọi các hàm để lấy metadata, description, và promotions
            metadata = get_single_item_metadata(href)
            description = get_single_desc_item_data(href)
            promotions = get_single_promotion_data(href)

            # Thêm dữ liệu sản phẩm vào danh sách
            product_data.append({
                "id": id_counter,
                "product": title,
                "original_price": original_price_value,
                "current_price": current_price_value,
                "discount_price": discount_price_value,
                "metadata": metadata,
                "description": description,
                "promotions": promotions
            })
            id_counter += 1  # Tăng id cho sản phẩm tiếp theo

            # Đợi một chút để tránh bị chặn khi truy xuất dữ liệu nhanh
            time.sleep(1)  # Đợi 1 giây giữa các sản phẩm

        page += 1

    # Chuyển danh sách dữ liệu thành DataFrame
    df = pd.DataFrame(product_data)
    return df

# Gọi hàm và lưu kết quả vào DataFrame
product_df = spider(3)  # Chỉ lấy 3 trang để thử nghiệm
print(product_df.head())


   id                            product  original_price  current_price  \
0   1  Samsung Galaxy Z Fold6 12GB/256GB        43990000       41990000   
1   2  Samsung Galaxy Z Fold6 12GB/512GB        47990000       45990000   
2   3    Samsung Galaxy Z Fold6 12GB/1TB        54990000       52990000   
3   4  Samsung Galaxy Z Flip6 12GB/256GB        28990000       26990000   
4   5  Samsung Galaxy Z Flip6 12GB/512GB        32990000       30990000   

   discount_price                                           metadata  \
0         2000000  {'Màn hình': {'Công nghệ màn hình': 'Dynamic A...   
1         2000000  {'Màn hình': {'Công nghệ màn hình': 'Dynamic A...   
2         2000000  {'Màn hình': {'Công nghệ màn hình': 'Dynamic A...   
3         2000000  {'Màn hình': {'Tần số quét (Hz)': '1-120Hz', '...   
4         2000000  {'Màn hình': {'Tần số quét (Hz)': '1-120Hz', '...   

                                         description  \
0  Samsung Galaxy Z Fold6 là dòng điện thoại thôn...   
1  S

In [15]:
product_df.head(1)

,id,product,original_price,current_price,discount_price,metadata,description,promotions
0,1,Samsung Galaxy Z Fold6 12GB/256GB,43990000,41990000,2000000,{'Màn hình': {'Công nghệ màn hình': 'Dynamic A...,Samsung Galaxy Z Fold6 là dòng điện thoại thôn...,"Trả góp tới 06 tháng không lãi suất, trả trước..."


In [14]:
from google.colab import files

# Lưu DataFrame vào file CSV
product_df.to_csv('product_data_3_pages_hoangha.csv', index=False, encoding='utf-8-sig')

# Download tập dữ liệu
files.download('product_data_3_pages_hoangha.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Convert Data to Json Format:

In [19]:
def convert_raw_data_to_string(row):
    # Chuyển đổi một hàng của DataFrame thành chuỗi, bao gồm metadata
    return (f"Tên sản phẩm: {row['product']}, "
            f"Giá gốc: {row['original_price']}, "
            f"Tiền giảm: {row['discount_price']}, "
            f"Mô tả: {row['description']}, "
            f"Metadata: {row['metadata']}, "
            f"Chương trình giảm giá: {row['promotions']}")

def process_and_save_data_from_df(product_df, output_file):
    schema = schema_english  # Hoặc schema_vietnamese nếu bạn muốn dùng schema tiếng Việt

    processed_data = []

    for index, row in product_df.iterrows():
        # Chuyển đổi hàng DataFrame thành chuỗi
        raw_data = convert_raw_data_to_string(row)

        # Gọi hàm convert_to_json để chuyển đổi
        json_data = convert_to_json(raw_data, schema)

        if json_data:
            processed_data.append(json_data)

    # Lưu dữ liệu đã xử lý vào tệp JSON
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(processed_data, f, ensure_ascii=False, indent=4)

# Gọi hàm để lưu dữ liệu vào JSON (sử dụng 10 hàng để test)
process_and_save_data_from_df(product_df.head(10), 'product_data_3_pages_hoangha.json')

In [20]:
import json

# Hàm đọc và in nội dung JSON với định dạng dễ đọc
def pretty_print_json(file_path):
    # Đọc dữ liệu từ file JSON
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # In dữ liệu với định dạng pretty print
    print(json.dumps(data, ensure_ascii=False, indent=4))

# Sau khi đã lưu file product_data_3_pages_hoangha.json, bạn có thể gọi hàm để in
pretty_print_json('product_data_3_pages_hoangha.json')


[
    {
        "general information": {
            "name": "Samsung Galaxy Z Fold6 12GB/256GB",
            "brand": "Samsung",
            "category": "Smartphone",
            "original_price": 43990000.0,
            "discount_price": 41990000.0,
            "description": "Samsung Galaxy Z Fold6 là dòng điện thoại thông minh thế hệ Galaxy Z Fold tiếp theo được Samsung công bố ra thị trường với nhiều tính năng và cập nhật lớn. Phiên bản này sẽ có màn hình ngoài 6.3 inch, PIN 4400mAh, và camera trước 10 MP. Galaxy Z Fold6 còn được trang bị hệ thống làm mát lớn hơn với công nghệ chipset Snapdragon 8 Gen 3 For Galaxy cực mạnh.",
            "design": "Thiết kế đa năng, màn hình gập",
            "promotion": "Giảm giá 2,000,000 VNĐ"
        },
        "technical specifications": {
            "display": [
                "Màn hình chính: 7.6 inch Dynamic AMOLED 2X, độ phân giải 2160 x 1856 pixels, tần số quét 120Hz",
                "Màn hình phụ: 6.3 inch, độ phân giải 968 x 2376 pi